# Load knowledge to neo4j

In [1]:
# %pip install --upgrade py2neo

Note: you may need to restart the kernel to use updated packages.


In [1]:
from py2neo import Graph
from py2neo import Node, Relationship
import csv
import json

def when(condition, true, false):
  return true if condition else false

In [2]:
graph = Graph('bolt://localhost:7687', auth=('neo4j','200Watt!'))

In [181]:
# graph.delete_all()

In [3]:
def capsnake(s: str):
  return "".join([c if c.isalnum() else "_" for c in s]).upper()

def rel(e, nodes):
  s = nodes[e['source']]
  t = nodes[e['target']]
  return Relationship(s, capsnake(e['interaction']), t)


In [183]:
with open('metamodel/maintainability-0-nodes.csv') as csv_file:
  csv_reader = csv.DictReader(csv_file)
  mm_nodes = {
    row['id']: Node(row['type'], name=row['name'])
    for row in csv_reader
  }

with open('metamodel/maintainability-0-edges.csv') as csv_file:
  csv_reader = csv.DictReader(csv_file)
  mm_edges = [rel(row, mm_nodes) for row in csv_reader]

print(mm_edges[123])

(Intercepting Filter)-[:EMPLOYS {}]->(Split module)


In [184]:
tx = graph.begin()

for node in mm_nodes.values():
  tx.create(node)

for edge in mm_edges:
  tx.create(edge)

graph.commit(tx)

In [12]:
prj = Node("Project", name='JHotDraw', version='5.1')
pkgs = dict()
prj_nodes = dict()
prj_edges = []
containment = []

with open('cases/JHotDraw-v5.1/facts/impl-time/jhotdraw-nodes.csv') as csv_file:
  csv_reader = csv.DictReader(csv_file)
  for row in csv_reader:
    if row['package'] not in pkgs:
      row['package'] = Node('Component', name=row['package'], type='PKG')
    pkg = row['package']
    clss = Node(row['type'], name=row['name'], fullname=row['id'])
    prj_nodes[row['id']] = clss
    containment.append(Relationship(pkg, "CONTAINS", clss))
    containment.append(Relationship(clss, "BELONGS_TO", prj))
  
print(containment[0], containment[1])

with open('cases/JHotDraw-v5.1/facts/impl-time/jhotdraw-edges.csv') as csv_file:
  csv_reader = csv.DictReader(csv_file)
  for row in csv_reader:
    relt = rel(row, prj_nodes)
    relt['weight'] = row['weight']
    prj_edges.append(relt)

print(prj_edges[123])

(CH.ifa.draw.applet)-[:CONTAINS {}]->(DrawApplet) (DrawApplet)-[:BELONGS_TO {}]->(JHotDraw)
(PolygonHandle)-[:SPECIALIZES {weight: '1'}]->(AbstractHandle)


In [186]:
tx = graph.begin()

tx.create(prj)

for node in pkgs.values():
  tx.create(node)

for node in prj_nodes.values():
  tx.create(node)

for edge in containment:
  tx.create(edge)

for edge in prj_edges:
  tx.create(edge)

graph.commit(tx)

In [187]:
adapter_pattern = graph.nodes.match('Pattern', name='Adapter').first()

print(adapter_pattern)

with open('metamodel/adapter-pattern-0-nodes.csv') as csv_file:
  csv_reader = csv.DictReader(csv_file)
  adp_nodes = { row['id']:  when(graph.nodes.match(row['type'], name=row['name']),
                                true = graph.nodes.match(row['type'], name=row['name']).first(),
                                false = Node(row['type'], name=row['name']))
    for row in csv_reader
  }

with open('metamodel/adapter-pattern-0-edges.csv') as csv_file:
  csv_reader = csv.DictReader(csv_file)
  adp_edges = [rel(row, adp_nodes) for row in csv_reader]

print(adp_nodes, adp_edges)

(_388:Pattern {name: 'Adapter'})
{'n0': Node('Role', name='Client'), 'n1': Node('Pattern', name='Adapter'), 'n2': Node('Role', name='Target'), 'n3': Node('Role', name='Adapter'), 'n4': Node('Role', name='Adaptee')} [INVOLVES(Node('Pattern', name='Adapter'), Node('Role', name='Client')), INVOLVES(Node('Pattern', name='Adapter'), Node('Role', name='Target')), INVOLVES(Node('Pattern', name='Adapter'), Node('Role', name='Adapter')), INVOLVES(Node('Pattern', name='Adapter'), Node('Role', name='Adaptee'))]


In [188]:
tx = graph.begin()

for node in adp_nodes.values():
  tx.create(node)

for edge in adp_edges:
  tx.create(edge)

graph.commit(tx)

In [4]:
with open('cases/JHotDraw-v5.1/patterns/ptidej-jhotdraw-v5.1.json') as json_file:
  data = json.load(json_file)


Reminder: flattening list in Python:

```python
flat_list = [
  item 
  for sublist in regular_list 
  for item in sublist
]
```

In [5]:
pattern_instances = [
  (microArchs['_name'], [(x,y) for x,sublist in [
    (role.title(), [actor['entity'] for actor in actors[role]])
    for actors in instance['roles'].values() 
    for role in actors
  ] for y in sublist])
  for microArchs in data['designPattern'] 
  for instance in microArchs['microArchitectures']['microArchitecture']
]

pattern_instances


[('Adapter',
  [('Client', 'CH.ifa.draw.standard.StandardDrawingView'),
   ('Client', 'CH.ifa.draw.standard.HandleTracker'),
   ('Target', 'CH.ifa.draw.framework.Handle'),
   ('Adapter', 'CH.ifa.draw.standard.ChangeConnectionEndHandle'),
   ('Adapter', 'CH.ifa.draw.standard.ChangeConnectionStartHandle'),
   ('Adapter', 'CH.ifa.draw.figures.ElbowHandle'),
   ('Adapter', 'CH.ifa.draw.standard.LocatorHandle'),
   ('Adapter', 'CH.ifa.draw.standard.ConnectionHandle'),
   ('Adapter', 'CH.ifa.draw.standard.EastHandle'),
   ('Adapter', 'CH.ifa.draw.figures.FontSizeHandle'),
   ('Adapter', 'CH.ifa.draw.standard.NorthEastHandle'),
   ('Adapter', 'CH.ifa.draw.standard.NorthHandle'),
   ('Adapter', 'CH.ifa.draw.standard.NorthWestHandle'),
   ('Adapter', 'CH.ifa.draw.standard.NullHandle'),
   ('Adapter', 'CH.ifa.draw.figures.GroupHandle'),
   ('Adapter', 'CH.ifa.draw.figures.PolyLineHandle'),
   ('Adapter', 'CH.ifa.draw.standard.SouthEastHandle'),
   ('Adapter', 'CH.ifa.draw.standard.SouthHandle'),

In [9]:
pattern_names = [
  # 'Adapter',
  'Strategy',
]

for p,i in pattern_instances:
  if p in pattern_names:
    roles = set(r for r,_ in i)
    for role in roles:
      if not graph.nodes.match('Role', name=role) and graph.nodes.match('Pattern', name=p):
        role_node = Node('Role', name=role)
        pat_node = graph.nodes.match('Pattern', name=p).first()
        role_edge = Relationship(pat_node, "INVOLVES", role_node)
        tx = graph.begin()
        tx.create(role_node)
        tx.create(role_edge)
        graph.commit(tx)


In [10]:
pi_nodes = []
pi_edges = []
for p,i in pattern_instances:
  if p in pattern_names:
    adp = graph.nodes.match('Pattern', name=p).first()
    pi = Node('PatternInstance')
    relpi = Relationship(pi, "INSTANCE_OF", adp)
    pi_nodes.append(pi)
    pi_edges.append(relpi)
    for role, clss in i:
      role_node = graph.nodes.match('Role', name=role).first()
      struct_node = graph.nodes.match('Structure', fullname=clss).first()
      pi_edges.append(Relationship(struct_node, "ACTS_AS", role_node))
      pi_edges.append(Relationship(pi, "INVOLVES", struct_node))

pi_nodes, pi_edges


([Node('PatternInstance'),
  Node('PatternInstance'),
  Node('PatternInstance'),
  Node('PatternInstance')],
 [INSTANCE_OF(Node('PatternInstance'), Node('Pattern', name='Strategy')),
  ACTS_AS(Node('Structure', fullname='CH.ifa.draw.figures.LineConnection', name='LineConnection'), Node('Role', name='Context')),
  INVOLVES(Node('PatternInstance'), Node('Structure', fullname='CH.ifa.draw.figures.LineConnection', name='LineConnection')),
  ACTS_AS(Node('Structure', fullname='CH.ifa.draw.contrib.PolygonFigure', name='PolygonFigure'), Node('Role', name='Context')),
  INVOLVES(Node('PatternInstance'), Node('Structure', fullname='CH.ifa.draw.contrib.PolygonFigure', name='PolygonFigure')),
  ACTS_AS(Node('Structure', fullname='CH.ifa.draw.figures.EllipseFigure', name='EllipseFigure'), Node('Role', name='Context')),
  INVOLVES(Node('PatternInstance'), Node('Structure', fullname='CH.ifa.draw.figures.EllipseFigure', name='EllipseFigure')),
  ACTS_AS(Node('Structure', fullname='CH.ifa.draw.figures

In [11]:
tx = graph.begin()

for node in pi_nodes + pi_edges:
  tx.create(node)

graph.commit(tx)